In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import numpy as np
import shutil
import os
!pip install wandb
import wandb
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('./drive/MyDrive/Colab Notebooks/CS6910/Part A/')

     |████████████████████████████████| 2.1MB 5.2MB/s 
     |████████████████████████████████| 163kB 21.2MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
     |████████████████████████████████| 133kB 28.1MB/s 
     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=a6ba030a1c27ce42d1a41afdf7f2bebf22122bfd4c3ef63f10c6d12b0494a175
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=3008bcfac1d706a7f4242b0d9edadabae4fc19c8913610e9a7abddbc01dd990d
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
Mounted at /content/drive


In [ ]:
!pip install wandb --upgrade

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.26)


In [ ]:
from torchvision.transforms import RandomCrop, RandomResizedCrop, RandomHorizontalFlip, Resize, CenterCrop, ToTensor, Normalize, Compose

## Dataset info
iNaturalist = {
    'Normalize': {
        'mean': (0.485, 0.456, 0.406),
        'std':  (0.229, 0.224, 0.225)
    }
}

## Dataloaders
def data_loader(train_data, val_data, test_data, batchSize):
    train_dataLoader = torch.utils.data.DataLoader(train_data, batch_size=batchSize, shuffle=True)
    val_dataLoader = torch.utils.data.DataLoader(val_data, batch_size=batchSize, shuffle=True)
    test_dataLoader = torch.utils.data.DataLoader(test_data, batch_size=batchSize, shuffle=False)
    loaders = {
        'train' : train_dataLoader,
        'valid' : val_dataLoader,
        'test'  : test_dataLoader
    }
    return loaders


## transforms to match model input dims
def transform():
    
    resize = 224     #128 #32
    val_resize = 256 #134 #36
    val_center_crop = resize
    
    train_t = Compose([RandomResizedCrop(resize),
                       RandomHorizontalFlip(),
                       ToTensor(),
                       Normalize(**iNaturalist['Normalize'])])
    valid_t = Compose([Resize(val_resize),
                       CenterCrop(resize),
                       ToTensor(),
                       Normalize(**iNaturalist['Normalize'])])
    test_t = Compose([Resize((resize,resize)), 
                      ToTensor(), 
                      Normalize(**iNaturalist['Normalize'])])
    
    transforms = {
        'training':   train_t,
        'validation': valid_t,
        'test': test_t
    }
    
    return transforms

## Load dataset fn
def load_datasets():
    transforms=transform()
    trainset  = torchvision.datasets.ImageFolder('./drive/MyDrive/Colab Notebooks/CS6910/Part A/inaturalist_12K/train_val/train', transforms['training'])
    valset    = torchvision.datasets.ImageFolder('./drive/MyDrive/Colab Notebooks/CS6910/Part A/inaturalist_12K/train_val/val', transforms['validation'])
    testset   = torchvision.datasets.ImageFolder('./drive/MyDrive/Colab Notebooks/CS6910/Part A/inaturalist_12K/val', transforms['test'])
    
    return trainset, valset, testset

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
if use_cuda == True:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
batch_size = 64
datasetTrain, datasetVal, datasetTest = load_datasets()
loaders = data_loader(datasetTrain, datasetVal, datasetTest, batch_size)

In [ ]:
modelName = 'CNN_5Layers_iNaturalist'

In [ ]:
def act_fn(act_name):
    if act_name=="relu":
        return nn.ReLU(inplace=True)
    
    elif act_name=="sigmoid":
        return nn.Sigmoid(inplace=True)
    
    elif act_name=="tanh":
        return nn.Tanh(inplace==True)

In [ ]:
class conv_block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size = 3 ,BN=True , NL="relu", stride = 1, padding = 0):
        super(conv_block, self).__init__()
        self.BN=BN
        self.NL=NL
        k = kernel_size
        
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size = k, stride = stride, padding = padding, bias=False)
        
        if self.BN==True:
            self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
            
        self.act = act_fn(NL)
        
    def forward(self, x):
        x = self.conv(x)
        
        if self.BN==True:
            x = self.bn(x)
        
        return self.act(x)

In [ ]:
class fc_block(nn.Module):
    def __init__(self, in_channels, out_channels, BN=False , NL="relu"):
        super(fc_block, self).__init__()
        self.BN=BN
        self.NL=NL
        self.fc = nn.Linear(in_channels, out_channels)
        
        if self.BN==True:
            self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
            
        self.act = act_fn(NL)
        
    def forward(self, x):
        x = self.fc(x)
        
        if self.BN==True:
            x = self.bn(x)
        
        x = self.act(x)
        
        return x

In [ ]:
def get_fc_in(input_dim, kernel_size_list, no_kernel_list):
    H = input_dim
    fc_in = H - kernel_size_list[0] + 1 # conv1
    fc_in = (fc_in - 2) //2  + 1 # max pool 1
    fc_in = fc_in - kernel_size_list[1] + 1 # conv2
    fc_in = (fc_in - 2) //2  + 1 # max pool 2
    fc_in = fc_in - kernel_size_list[2] + 1 #conv3
    fc_in = (fc_in - 2) //2  + 1 # max pool 3
    fc_in = fc_in - kernel_size_list[3] + 1 #conv4
    fc_in = (fc_in - 2) //2  + 1 # max pool 4
    fc_in = fc_in - kernel_size_list[4] + 1 #conv5
    fc_in = (fc_in - 2) //2  + 1 # max pool 5
    #print(fc_in)
    return fc_in * fc_in * no_kernel_list[4]

In [ ]:
class CNN_5layer(nn.Module):
    def __init__(self, kernel_size_list, no_kernel_list, act_fn_dict, dropout_list, fc1_nodes, no_classes):
        super(CNN_5layer, self).__init__()
        self.dropout_list = dropout_list
        self.input_dim = 224
        #self.input_dim = 128
        self.conv1 = conv_block(3, no_kernel_list[0], kernel_size=kernel_size_list[0], BN=False, NL=act_fn_dict['conv1'])
        self.maxpool1 = nn.MaxPool2d((2, 2))
        self.conv2 = conv_block(no_kernel_list[0], no_kernel_list[1], kernel_size=kernel_size_list[1], BN=True, NL=act_fn_dict['conv2'])
        self.maxpool2 = nn.MaxPool2d((2, 2))
        
        if self.dropout_list[0]!=0:
            self.dropout1 = nn.Dropout(dropout_list[0])

        self.conv3 = conv_block(no_kernel_list[1], no_kernel_list[2], kernel_size=kernel_size_list[2], BN=True, NL=act_fn_dict['conv3'])
        self.maxpool3 = nn.MaxPool2d((2, 2))
        self.conv4 = conv_block(no_kernel_list[2], no_kernel_list[3], kernel_size=kernel_size_list[3], BN=True, NL=act_fn_dict['conv4'])
        self.maxpool4 = nn.MaxPool2d((2, 2))
        
        if self.dropout_list[1]!=0:
            self.dropout2 = nn.Dropout(dropout_list[1])

        self.conv5 = conv_block(no_kernel_list[3], no_kernel_list[4], kernel_size=kernel_size_list[4], BN=True, NL=act_fn_dict['conv5'])
        self.maxpool5 = nn.MaxPool2d((2, 2))
        
        self.fc1_in_features = get_fc_in(self.input_dim, kernel_size_list, no_kernel_list)
        
        self.fc1 = fc_block(self.fc1_in_features, fc1_nodes , NL=act_fn_dict['fc1'])
        
        if self.dropout_list[2]!=0:
            self.dropout3 = nn.Dropout(dropout_list[2])
        
        self.fc2 = nn.Linear(fc1_nodes, no_classes)
    
    
    def forward(self, x):
        if x.shape[2]!=self.input_dim:
            print("input dim not matched")
            return
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        if self.dropout_list[0]!=0:
            x = self.dropout1(x)
        
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.conv4(x)
        x = self.maxpool4(x)
        
        if self.dropout_list[1]!=0:
            x = self.dropout2(x)
        
        x = self.conv5(x)
        x = self.maxpool5(x)
        
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        if self.dropout_list[2]!=0:
            x = self.dropout3(x)
        
        x = self.fc2(x)
        
        return x

In [ ]:
def train(start_epochs, n_epochs, valid_loss_min_input, loaders, model, optimizer, criterion,scheduler, use_cuda):
    valid_loss_min = valid_loss_min_input 
    
    for epoch in range(start_epochs, start_epochs+n_epochs):
        
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        tnum_correct = 0
        tnum_examples = 0
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
                
            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, target)
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],target).view(-1)
            tnum_correct += torch.sum(correct).item()
            tnum_examples += correct.shape[0]
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
        train_acc = tnum_correct / tnum_examples
        
        ######################    
        # validate the model #
        ######################
        model.eval()
        num_correct = 0
        num_examples = 0
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],target).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
            
        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        valid_acc = num_correct / num_examples
        scheduler.step()
        
        print('Epoch: {}\tTraining Loss: {:.6f}\tTrain Accuracy: {:.2f}\tValidation Loss: {:.6f}\tvalidation Accuracy: {:.2f}'.format(
            epoch, 
            train_loss,
            train_acc,
            valid_loss,
            valid_acc
            ))
        
        wandb.log({'epoch': epoch,'train loss': train_loss,'train accuracy': train_acc,
                   'val loss': valid_loss, 'val accuracy': valid_acc})
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f})'.format(valid_loss_min,valid_loss))
            valid_loss_min = valid_loss
            
    return model

In [ ]:
def config_str_list_int(s):
    l=list(map(int, s[3:].split('-')))
    return l

def config_str_list_float(s):
    l=list(map(float, s.split('-')))
    return l

In [ ]:
def sp_train():
    config_defaults = {
        'epochs': 25,
        'kernel_size_config':'1) 5-5-3-3-3' ,
        'no_kernel_config':'1) 32-32-32-32-32',
        'dropout_config':'0-0-0.4',
        'fc1_nodes': 512,
        'batch_size': 64
    }
    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    run_name="kSizes:["+config.kernel_size_config+"] kNumbers:["+config.no_kernel_config+"] dp:["+config.dropout_config+"] fc1:["+str(config.fc1_nodes)+"] bs:["+str(config.batch_size)+"]"
    wandb.run.name=run_name
    
    act_fn_dict = {
        'conv1':'relu',
        'conv2':'relu',
        'conv3':'relu',
        'conv4':'relu',
        'conv5':'relu',
        'fc1':'relu'
    }
    
    kernel_size_list = config_str_list_int(config.kernel_size_config)
    no_kernel_list = config_str_list_int(config.no_kernel_config)
    dropout_list = config_str_list_float(config.dropout_config)
    fc1_nodes = config.fc1_nodes
    no_classes = 10
    
    model = CNN_5layer(kernel_size_list, no_kernel_list, act_fn_dict, dropout_list, fc1_nodes, no_classes)
    model = model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)
    criterion = nn.CrossEntropyLoss()
    
    trained_model = train(start_epochs = 1,
                      n_epochs = config.epochs,
                      valid_loss_min_input = np.Inf,
                      loaders = loaders,
                      model = model,
                      optimizer = optimizer,
                      criterion = criterion,
                      scheduler = scheduler,
                      use_cuda = use_cuda
                     )

In [ ]:
sweep_config = {
    'method': 'grid', 
    'metric': {
      'name': 'val accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values':[25]
        },
        'kernel_size_config':{
            'values': ['1) 5-5-3-3-3', 
                       '2) 3-3-3-3-3', 
                       '3) 3-3-3-5-5'
                       ]
        },
        'no_kernel_config': {
            'values':['1) 32-32-32-32-32', 
                      '2) 64-64-32-32-16', 
                      '3) 32-32-64-64-128'] 
        },
        'dropout_config':{
            'values':['0-0-0.5']
        },
        'fc1_nodes':{
            'values': [512,1024]
        },
        'batch_size': {
            'values':[64]
        }
    }
}

In [ ]:
#sweep_id = wandb.sweep(sweep_config, project=modelName+"_Sweep_1")

In [ ]:
#wandb.agent(sweep_id, sp_train)

In [ ]:
wandb.agent(sweep_id = 'm9juzani', project=modelName+"_Sweep_1", function = sp_train)

wandb: Agent Starting Run: ohyvu2tr with config:
wandb: 	batch_size: 64
wandb: 	dropout_config: 0-0-0.5
wandb: 	epochs: 25
wandb: 	fc1_nodes: 1024
wandb: 	kernel_size_config: 3) 3-3-3-5-5
wandb: 	no_kernel_config: 3) 32-32-64-64-128
wandb: Currently logged in as: rayanz (use `wandb login --relogin` to force relogin)


Epoch: 1	Training Loss: 0.000244	Train Accuracy: 0.19	Validation Loss: 0.002030	validation Accuracy: 0.26
Validation loss decreased (inf --> 0.002030)
